In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [ ]:
# columns = ["_time", "_measurement", "packet_interarrival_time", "packet_jitter_weighted", "packet_owd"]
num_cols = ["packet_interarrival_time", "packet_jitter_weighted", "packet_owd"]
str_cols = ["_time", "_measurement"]
columns = num_cols + str_cols
# df = pd.read_csv("cbr_low_bw_med_dur_low_pl_1_analysis.csv", skiprows=3)[columns]
# df.head()

In [ ]:
def get_aggregated_df(df, point_lim):
    group_size = df.shape[0] // point_lim 
    df['group'] = pd.Series(range(df.shape[0])) // group_size

    aggregated_mean_df = df.groupby('group')[num_cols].mean().rename(columns={col: f"{col}_mean" for col in num_cols})
    aggregated_std_df = df.groupby('group')[num_cols].std().rename(columns={col: f"{col}_std" for col in num_cols})
    aggregated_str_df = df.groupby('group')[str_cols].first()

    aggregated_df = pd.concat([aggregated_mean_df, aggregated_std_df, aggregated_str_df], axis=1).reset_index(drop=True)

    aggregated_df['timestamp'] = pd.to_datetime(aggregated_df['_time'], format="ISO8601")
    aggregated_df.set_index('timestamp', inplace=True)

    min_ts = aggregated_df.index.min()

    aggregated_df.index = (aggregated_df.index - min_ts).total_seconds()

    return aggregated_df

In [ ]:
POINT_LIM = 250

In [ ]:
# IRRELEVANT 
fig = go.Figure()

x = aggregated_df.index.values
y = aggregated_df['packet_owd_mean']
y_upper = aggregated_df['packet_owd_std']
y_lower = aggregated_df['packet_owd_std']

fig.add_traces(go.Scatter(
    x=x,
    y=y+y_upper,
    # line_color="indigo",
    mode="lines"
))

fig.add_traces(go.Scatter(
    x=x,
    y=y-y_lower,
    # line_color="",
    mode="lines",
    fill="tonexty"
))

fig.add_traces(go.Scatter(
    x=x,
    y=y,
    line_color="green",
    mode="lines"
))

# fig.add_traces(go.Scatter(x=x, y=aggregated_df['packet_owd_mean']))

fig.show()

In [ ]:
df1, df2, df3 = \
    pd.read_csv("cbr_1000pps/cbr_low_bw_med_dur_low_pl_1_analysis.csv", skiprows=3)[columns],\
    pd.read_csv("cbr_1000pps/cbr_low_bw_med_dur_low_pl_2_analysis.csv", skiprows=3)[columns],\
    pd.read_csv("cbr_1000pps/cbr_low_bw_med_dur_low_pl_3_analysis.csv", skiprows=3)[columns]

agg_df1, agg_df2, agg_df3 = \
    get_aggregated_df(df1, POINT_LIM),\
    get_aggregated_df(df2, POINT_LIM),\
    get_aggregated_df(df3, POINT_LIM)

In [ ]:
new_agg_df = pd.concat([agg_df1, agg_df2, agg_df3], axis=0)

In [ ]:
fig = px.line(x=new_agg_df.index, y=new_agg_df['packet_owd_mean'], color=new_agg_df['_measurement'], template='simple_white')

fig.update_xaxes(
    title="Timestamp (s)",
    showline=True,
    linecolor="black",
    linewidth=1,
    mirror=True,
    gridcolor="LightGray",
    showgrid=True
)
fig.update_yaxes(
    title="One-way delay (s)",
    showline=True,
    linecolor="black",
    linewidth=1,
    mirror=True,
    gridcolor="LightGray",
    showgrid=True
)

fig.update_layout(
    legend_title_text="Run index"
)

newLegendNames = {
    'cbr_low_bw_med_dur_low_pl_1_analysis': '1',
    'cbr_low_bw_med_dur_low_pl_2_analysis': '2',
    'cbr_low_bw_med_dur_low_pl_3_analysis': '3'
}

fig.for_each_trace(lambda t: t.update(name=newLegendNames[t.name]))

fig.show()

In [ ]:
px.line(x=new_agg_df.index, y=new_agg_df['packet_jitter_weighted_mean'], color=new_agg_df['_measurement'])

In [ ]:
agg_columns = ['_time', '_measurement', 'packet_loss', 'throughput']
analysis_aggregate_df_1, analysis_aggregate_df_2, analysis_aggregate_df_3 = \
    pd.read_csv("cbr_1000pps/cbr_low_bw_med_dur_low_pl_1_analysis_aggregate.csv", skiprows=3)[agg_columns],\
    pd.read_csv("cbr_1000pps/cbr_low_bw_med_dur_low_pl_2_analysis_aggregate.csv", skiprows=3)[agg_columns],\
    pd.read_csv("cbr_1000pps/cbr_low_bw_med_dur_low_pl_3_analysis_aggregate.csv", skiprows=3)[agg_columns]

analysis_aggregate_df_1['timestamp'] = pd.to_datetime(analysis_aggregate_df_1['_time'], format="ISO8601")
analysis_aggregate_df_1.set_index('timestamp', inplace=True)
min_ts = analysis_aggregate_df_1.index.min()
analysis_aggregate_df_1.index = (analysis_aggregate_df_1.index - min_ts).total_seconds()

analysis_aggregate_df_2['timestamp'] = pd.to_datetime(analysis_aggregate_df_2['_time'], format="ISO8601")
analysis_aggregate_df_2.set_index('timestamp', inplace=True)
min_ts = analysis_aggregate_df_2.index.min()
analysis_aggregate_df_2.index = (analysis_aggregate_df_2.index - min_ts).total_seconds()

analysis_aggregate_df_3['timestamp'] = pd.to_datetime(analysis_aggregate_df_3['_time'], format="ISO8601")
analysis_aggregate_df_3.set_index('timestamp', inplace=True)
min_ts = analysis_aggregate_df_3.index.min()
analysis_aggregate_df_3.index = (analysis_aggregate_df_3.index - min_ts).total_seconds()

new_agg_df_aggregate = pd.concat([analysis_aggregate_df_1, analysis_aggregate_df_2, analysis_aggregate_df_3], axis=0)

In [ ]:
fig = px.line(x=new_agg_df_aggregate.index, y=new_agg_df_aggregate['packet_loss'], color=new_agg_df_aggregate['_measurement'], template='simple_white')

fig.update_xaxes(
    title="Timestamp (s)",
    showline=True,
    linecolor="black",
    linewidth=1,
    mirror=True,
    gridcolor="LightGray",
    showgrid=True
)
fig.update_yaxes(
    title="One-way delay (s)",
    showline=True,
    linecolor="black",
    linewidth=1,
    mirror=True,
    gridcolor="LightGray",
    showgrid=True
)

fig.update_layout(
    legend_title_text="Run index"
)

newLegendNames = {
    'cbr_low_bw_med_dur_low_pl_1_analysis_aggregate': '1',
    'cbr_low_bw_med_dur_low_pl_2_analysis_aggregate': '2',
    'cbr_low_bw_med_dur_low_pl_3_analysis_aggregate': '3'
}

fig.for_each_trace(lambda t: t.update(name=newLegendNames[t.name]))

fig.show()

In [ ]:
px.line(x=new_agg_df_aggregate.index, y=new_agg_df_aggregate['throughput'], color=new_agg_df_aggregate['_measurement'])